# Исследование надёжности заёмщиков

**Цель исследования:** определить влияние семейного положения и количества детей клиента на факт погашения кредита в срок.

**Используемые данные**: статистика банка о платёжеспособности клиентов.

**Описание данных**

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Оглавление

1. [Общая информация о данных](#1)  
2. [Предобработка данных](#2)
<br> 2.1. [Обработка пропусков](#2.1)
<br> 2.2. [Замена типов данных](#2.2)
<br> 2.3. [Обработка дубликатов](#2.3)
<br> 2.4. [Лемматизация](#2.4)
<br> 2.5. [Категоризация данных](#2.5)
3. [Ответы на вопросы](#3)
4. [Заключение](#4)

<a id="1"></a>
## 1. Общая информация о данных

Импортируем библиотеку `pandas` и открываем файл с данными. Посмотрим на общую информацию о наборе данных:

In [1]:
from pymystem3 import Mystem
import pandas as pd

data = pd.read_csv("/datasets/data.csv")
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим на строки с пропущенными значениями:

In [3]:
data[data["total_income"].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Похоже, что пропуски в разных столбцах взаимосвязаны: если пропущено значение в `days_employed`, то будет пропущено и значение в `total_income`. Можно проверить это предположение, воспользовавшись логической индексацией:

In [4]:
mask = (data["days_employed"].isna()) & (~data["total_income"].isna())
data[mask].shape[0]

0

Получили пустой dataframe. Вероятно, пропуски действительно связаны.

**Вывод**

Пропуски есть в двух столбцах: `days_employed` и `total_income`. Причём пропуски в `days_employed` связаны с пропусками в `total_income`, то есть у тех, кто не указал общий трудовой стаж, не указан и ежемесячный доход. 

<a id="2"></a>
## 2. Предобработка данных

<a id="2.1"></a>
### 2.1. Обработка пропусков

Можно предположить, что ежемесячный доход человека связан с его типом занятости. Будем заполнять пропуски в столбце `total_income` медианным значением для указанного типа занятости человека. Сначала посмотрим, какие есть типы занятости:

In [5]:
data["income_type"].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

И типы занятости у тех людей, чей ежемесячный доход не указан:

In [6]:
income_types_with_nan = data[data["total_income"].isna()]["income_type"].value_counts()
income_types_with_nan

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Убедимся, что в столбце `total_income` нет аномалий:

In [7]:
print("Минимальный ежемесячный доход равен: ", data["total_income"].min())
print("Максимальный ежемесячный доход равен:", data["total_income"].max())
print("Средний ежемесячный доход равен:     ", data["total_income"].mean())
print("Медианный ежемесячный доход равен:   ", data["total_income"].median())

Минимальный ежемесячный доход равен:  20667.26379327158
Максимальный ежемесячный доход равен: 2265604.028722744
Средний ежемесячный доход равен:      167422.30220817294
Медианный ежемесячный доход равен:    145017.93753253992


Значения не выглядят необычно, будем считать, что аномалий нет. Рассчитаем медианное значение ежемесячного дохода для каждой из этих категорий и заменим пропуски этими медианами:

In [8]:
medians = data.groupby("income_type")["total_income"].median()

for income_type in income_types_with_nan.index:
    mask = data["income_type"] == income_type
    data.loc[mask, "total_income"] = data.loc[mask, "total_income"].fillna(medians[income_type])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропущенных значений в `total_income` больше нет. 

Если обратить внимание на `days_employed`, то можно заметить, что значения выглядят странно: у многих клиентов стаж принимает отрицательное значение, а у некоторых принимает очень большое положительное значение:

In [10]:
data["days_employed"].sort_values(ascending=False)[:5]

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
Name: days_employed, dtype: float64

In [11]:
data[data["days_employed"] > 0]["days_employed"].sort_values(ascending=True)[:5]

20444    328728.720605
9328     328734.923996
17782    328771.341387
14783    328795.726728
7229     328827.345667
Name: days_employed, dtype: float64

В таком виде использовать эти данные для замены пропущенных значений нельзя. Вероятно, стаж клиентов записан не в днях, а в часах. Поделим значение на 24 часа:

In [12]:
mask = data["days_employed"] > 0
data.loc[mask, "days_employed"] = data.loc[mask, "days_employed"]/24

In [13]:
data[data["days_employed"] > 0]["days_employed"].sort_values(ascending=True)[:5]

20444    13697.030025
9328     13697.288500
17782    13698.805891
14783    13699.821947
7229     13701.139403
Name: days_employed, dtype: float64

Значения стали похожи на остальные в этом столбце во абсолтюной величине, но отличаются знаком. Возьмём модуль:

In [14]:
data["days_employed"] = data["days_employed"].apply(abs)
data["days_employed"].sort_values()[:5]

17437    24.141633
8336     24.240695
6157     30.195337
9683     33.520665
2127     34.701045
Name: days_employed, dtype: float64

Всё готово для замены пропущенных значений в `days_employed`. Опять будем использовать медианные значения для каждого типа клиентов.

In [15]:
medians = data.groupby("income_type")["days_employed"].median()
medians

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

_(Любопытная особенность: у безработных очень большой стаж работы. Если посмотреть на эти записи, то их окажется ровно две. При этом стаж клиентов будет больше их возраста! Очевидно, что это артефакты, однако в задании эти данные использовать не предполагается, поэтому можно их не обрабатывать.)_

In [16]:
for income_type in income_types_with_nan.index:
    mask = data["income_type"] == income_type
    data.loc[mask, "days_employed"] = data.loc[mask, "days_employed"].fillna(medians[income_type])

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Убедились, что пропущенных значений больше нет, переходим к замене типов данных в столбцах.

**Вывод**

Заменили пропущенные значения на медианные для соответствующих типов занятости. Обнаружили артефакты в данных и поправили некоторые из них. 

In [18]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,4600.535795,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,5354.354167,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.549623,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1993.522017,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<a id="2.2"></a>
### 2.2. Замена типа данных

Заменим тип данных `days_employed` и `total_income` на `int`, для этого воспользуемся методом `.astype()`.

In [19]:
data = data.astype({"days_employed": "int64", "total_income": "int64"})
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Заменили тип данных в выбранных столбцах на `int`.

<a id="2.3"></a>
### 2.3. Обработка дубликатов

Посмотрим на данные в столбцах со строковыми значениями (столбец `income_type` уже рассматривали выше):

In [20]:
data["education"].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [21]:
data["family_status"].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [22]:
data["purpose"].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце `education` есть одинаковые значения, но записанные в разном регистре, в остальных столбцах такого нет. Приведём данные к нижнему регистру, посмотрим на количество явных дубликатов и удалим их методом `drop_duplicates()`:

In [23]:
data["education"] = data["education"].str.lower()

In [24]:
print("Число явных дубликатов:", data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True);

Число явных дубликатов: 71


**Вывод**

Нашли явные дубликаты в данных и избавились от них.

<a id="2.4"></a>
### 2.4. Лемматизация

Если посмотреть на список значений, которые встречаются в `purpose`, то можно выделить четыре основных цели кредита: покупка/ремонт/строительство недвижимости, покупку автомобиля, получение образования и проведение свадьбы. Обозначим эти цели как категории: `недвижимость`, `автомобиль`, `образование`, `свадьба`.

Чтобы автоматически назначить категорию, нужно сначала провести лемматизацию слов, входящих в описание цели кредита, а затем проверить, попадает ли какое-либо ключевое слово (см. названия категорий) в результат лемматизации. Будем использовать для этого библиотеку `pymystem3`. Импортируем из библиотеки класс `Mystem`, затем создаём экземпляр этого класса, применяем его метод `.lemmatize()` к столбцу `purpose`. Результат сохраним в новый столбец `purpose_lemmatized`.

In [25]:
data.loc[:, "purpose_lemmatized"] = data["purpose"].apply(Mystem().lemmatize)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

Лемматизировали столбец с целями кредита.

<a id="2.5"></a>
### 2.5. Категоризация данных

Напишем функцию `assign_category()`, которая будет категоризировать цели клиентов, и применим её к столбцу `purpose_lemmatized`. Результат сохраняем в столбец `purpose_category`:

In [26]:
def assign_category(list_of_words):
    if "недвижимость" in list_of_words or "жилье" in list_of_words:
        return "недвижимость"
    elif "автомобиль" in list_of_words:
        return "автомобиль"
    elif "образование" in list_of_words:
        return "образование"
    elif "свадьба" in list_of_words:
        return "свадьба"
    else:
        None
        


data.loc[:, "purpose_category"] = data["purpose_lemmatized"].apply(assign_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


Убедимся, что в полученном столбце с категориями нет пропущенных значений и удалим ненужный столбец `purpose_lemmatized`:

In [27]:
print("Количество пропущенных значений:", data["purpose_category"].isna().sum())
data.drop("purpose_lemmatized", axis=1, inplace=True)

Количество пропущенных значений: 0


**Вывод**

Получили столбец с категориями целей кредита, проверили его на наличие пропущенных значений.

<a id="3"></a>
## 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим на данные в столбце `children`:

Там встречаются люди с 20 детьми, а также те, у кого -1 ребёнок. По всей видимости это ошибочные значения. Будем считать, что вместо `-1` должно быть `1`, а вместо `20` - значение `2`. 

In [28]:
data["children"].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [29]:
data["children"].replace({-1: 1, 20: 2}, inplace=True)
data["children"].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Теперь посчитаем (в процентах) доли людей с детьми и без детей, которые имели задолженность по кредиту:

In [30]:
data_pivot_children = data.pivot_table(index="children", values="debt", aggfunc=["sum", "count"])
data_pivot_children["debt, %"] = data_pivot_children["sum"]["debt"]/data_pivot_children["count"]["debt"] * 100
data_pivot_children

,sum,count,"debt, %"
,debt,debt,
children,,,
0,1063,14091,7.543822
1,445,4855,9.165808
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


**Вывод**

Полученные данные говорят о том, что клиенты без детей несколько реже имели задолженность по кредиту, чем клиенты с детьми. Однако разница не очень большая (в среднем чуть больше 1.5 п.п.).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим, как влияет семейное положение на возврат кредита в срок. Сразу сгруппируем данные по значениям столбца `family_status` и посчитаем доли:

In [31]:
data_pivot_family = data.pivot_table(index="family_status", values="debt", aggfunc=["sum", "count"])
data_pivot_family["debt, %"] = data_pivot_family["sum"]["debt"]/data_pivot_family["count"]["debt"] * 100
data_pivot_family

,sum,count,"debt, %"
,debt,debt,
family_status,,,
Не женат / не замужем,274,2810,9.750890
в разводе,85,1195,7.112971
вдовец / вдова,63,959,6.569343
гражданский брак,388,4151,9.347145
женат / замужем,931,12339,7.545182


**Вывод**

Чаще всего не возвращали кредит в срок неженатые (незамужние) люди и те, кто живут в "гражданском браке". Реже всех - вдовцы и вдовы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим на влияние уровня дохода. Получим статистическое описание столбца `total_income` с помощью метода `.describe()`, чтобы иметь возможность разбить клиентов по уровню их дохода на три категории: `низкий`, `средний` и `высокий`. Чтобы получить примерно равные по количеству группы клиентов, передадим аргумент `percentiles=[0.33, 0.67]`.

In [32]:
data.total_income.describe(percentiles=[0.33, 0.67])

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
33%      1.185140e+05
50%      1.425940e+05
67%      1.723570e+05
max      2.265604e+06
Name: total_income, dtype: float64

Получили, что 33-й процентиль равен примерно 119 тыс., 67-й - примерно 172 тыс. 

Напишем функцию, которая будет выдавать одну из трёх категорий уровня дохода в зависимости от дохода клиента, и применим её к столбцу `total_income`. Результат запишем в столбец `total_income_type`.

In [33]:
def total_income_type(row):
    if row <= 1.19 * 10**5:
        return "низкий"
    elif row <= 1.72 * 10**5:
        return "средний"
    else: 
        return "высокий"
    
    
data["total_income_type"] = data["total_income"].apply(total_income_type)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


In [34]:
data_pivot_income = data.pivot_table(index="total_income_type", values="debt", aggfunc=["sum", "count"])
data_pivot_income["debt, %"] = data_pivot_income["sum"]["debt"]/data_pivot_income["count"]["debt"] * 100
data_pivot_income

,sum,count,"debt, %"
,debt,debt,
total_income_type,,,
высокий,562,7551,7.442723
низкий,581,7128,8.150954
средний,598,6775,8.826568


**Вывод**

Люди с высоким доходом чуть реже имели задолженность по кредиту, однако отличие совсем небольшое и, скорее всего, статистически незначимое. _Возможно, имеет смысл использовать более мелкие категории, например, "доход до 50 тыс.", "доход от 50 до 100 тыс." и так далее._

- Как разные цели кредита влияют на его возврат в срок?

Группируем клиентов по цели кредита и считаем доли должников:

In [35]:
data_pivot_purpose = data.pivot_table(index="purpose_category", values="debt", aggfunc=["sum", "count"])
data_pivot_purpose["debt, %"] = data_pivot_purpose["sum"]["debt"]/data_pivot_purpose["count"]["debt"] * 100
data_pivot_purpose

,sum,count,"debt, %"
,debt,debt,
purpose_category,,,
автомобиль,403,4306,9.359034
недвижимость,782,10811,7.233373
образование,370,4013,9.220035
свадьба,186,2324,8.003442


**Вывод**

Чаще всего клиенты не возвращали кредиты, которые взяли для покупки автомобиля или получения образования, реже всего - для покупки недвижимости. 

In [36]:
data_pivot_income_purpose = data.pivot_table(index= [ 'total_income_type', 'purpose_category'], columns='debt', values='purpose', aggfunc='count')
data_pivot_income_purpose['ratio '] = (data_pivot_income_purpose[1] / (data_pivot_income_purpose[0]+data_pivot_income_purpose[1])) * 100

display(data_pivot_income_purpose)

debt                                   0    1     ratio 
total_income_type purpose_category                      
высокий           автомобиль        1377  132   8.747515
                  недвижимость      3602  263   6.804657
                  образование       1254  116   8.467153
                  свадьба            756   51   6.319703
низкий            автомобиль        1290  131   9.218860
                  недвижимость      3300  245   6.911142
                  образование       1223  132   9.741697
                  свадьба            734   73   9.045849
средний           автомобиль        1236  140  10.174419
                  недвижимость      3127  274   8.056454
                  образование       1166  122   9.472050
                  свадьба            648   62   8.732394

<a id="4"></a>
## 4. Заключение

Был проведён предварительный анализ набора данных, в ходе которого были обнаружены пропущенные и некорректные (артефакты) значения, дубликаты. Пропущенные значения были заменены на характерные, некорректные значения проанализированы и заменены, исходя из контекста, на наиболее подходящие. Дубликаты были удалены из набора данных. Проведена категоризация клиентов по целям кредита путём лемматизации. Проведена категоризация клиентов по уровню их дохода.

Анализ данных о клиентах показывает, что чаще всего кредиты не возвращают __не состоящие в официальном браке__ люди, у которых __есть дети__, а целью кредита является __покупка автомобиля__ или __получение образования__. При этом уровень дохода (при выбранном типе категоризации клиентов) не оказывает существенного значения на вероятность долга по кредиту.